# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
!pip install langchain_community

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [8]:
prompt = ChatPromptTemplate.from_template(
    "Describe the main features, advantages, and possible use cases of the product: {product_name}."
)

In [9]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\amjad\AppData\Local\Temp\ipykernel_24568\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Samsung Galaxy Tab S9"  # Example product
chain.run(product)

'The Samsung Galaxy Tab S9 is a high-end tablet from Samsung that comes with several impressive features and advantages. \n\nMain features:\n1. Display: The Galaxy Tab S9 features an immersive 12.4-inch Super AMOLED display with a resolution of 2800 x 1752 pixels, which provides stunning visuals and vibrant colors.\n2. Performance: The tablet is powered by the Qualcomm Snapdragon 865+ processor and comes with 8GB of RAM, which ensures smooth performance and multitasking capabilities.\n3. S Pen support: The Galaxy Tab S9 comes with S Pen support, allowing users to take notes, draw, and navigate the tablet with precision.\n4. Battery life: The tablet is equipped with a 10,090mAh battery that provides long-lasting usage on a single charge.\n5. Connectivity: The Galaxy Tab S9 supports 5G connectivity, allowing for faster download and streaming speeds.\n\nAdvantages:\n1. Large display: The large 12.4-inch display is great for multimedia consumption, gaming, and productivity tasks.\n2. Power

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
  "Given the product {product_name}, write a detailed description that highlights its key features and unique selling points."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
  "Based on the following product description, generate a compelling advertisement:\n\n{product_description}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The Samsung Galaxy Tab S9 is the latest addition to Samsung's line of high-end tablets, designed to offer users a premium experience with top-of-the-line features and performance. This sleek and stylish tablet boasts a stunning 12.4-inch Super AMOLED display with a resolution of 2800 x 1752 pixels, providing crisp and vibrant visuals for an immersive viewing experience.

One of the standout features of the Galaxy Tab S9 is its powerful performance capabilities. It is powered by the latest Qualcomm Snapdragon 895 processor, making it one of the fastest and most efficient tablets on the market. This ensures smooth multitasking, seamless streaming, and fast app launches, even with demanding tasks.

In terms of storage, the Galaxy Tab S9 comes with generous options, ranging from 128GB to 1TB of internal storage, so you can store all your photos, videos, and documents without worrying about running out of space. Additionally, the tablet suppor

"Introducing the Samsung Galaxy Tab S9 - the ultimate high-end tablet for those who demand the best. With a stunning 12.4-inch Super AMOLED display and lightning-fast Qualcomm Snapdragon 895 processor, this tablet delivers a premium experience like no other.\n\nStore all your photos, videos, and documents with ease thanks to the generous storage options ranging from 128GB to 1TB, plus support for up to 1TB microSD card expansion. Stay connected with 5G support for blazing-fast internet speeds and Wi-Fi 6E for enhanced connectivity and reduced latency.\n\nCapture memories on the go with the high-quality dual-camera system, featuring a 13MP main camera and a 5MP ultra-wide camera. And with a large 10,090mAh battery offering all-day usage on a single charge, you can enjoy your favorite content for hours on end.\n\nWhether you're a power user or a multimedia enthusiast, the Samsung Galaxy Tab S9 has everything you need and more. Upgrade to the ultimate tablet experience today."

**Repeat the above twice for different products**

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [19]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this customer review to English: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translation"#Give a name to your output
                    )


In [20]:
second_prompt = ChatPromptTemplate.from_template(
     "Summarize the following translated customer review: {translation}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [21]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following customer review into {target_language}: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_review"
                      )


In [22]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
         "Given the translated review:\n\n{translated_review}\n\n"
    "And its summary:\n\n{summary}\n\n"
    "Write a follow-up message to the customer that acknowledges their feedback and offers further help."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up"
                     )


In [23]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review", "target_language"],
    output_variables=["translated_review", "summary", "follow_up"],
    verbose=True
)

In [26]:
review = df.Review[5]
overall_chain.run({
    "review": review,
    "target_language": "English"
})

ValueError: `run` not supported when there is not exactly one output key. Got ['translated_review', 'summary', 'follow_up'].

**Repeat the above twice for different products or reviews**

## Router Chain

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [29]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [30]:
llm = ChatOpenAI(temperature=0)

In [31]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [32]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [33]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [34]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [35]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\amjad\AppData\Local\Temp\ipykernel_24568\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [36]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [37]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [38]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [41]:
# Arabic review example with correct method
arabic_review = "هذا المنتج رائع جدًا وساعدني كثيرًا في حياتي اليومية. أنصح الجميع باستخدامه."
result = overall_chain.invoke({
    "review": arabic_review,
    "target_language": "Arabic"
})

print("Translated Review:\n", result["translated_review"])
print("\nSummary:\n", result["summary"])
print("\nFollow-Up Message:\n", result["follow_up"])




> Entering new SequentialChain chain...

> Finished chain.
Translated Review:
 هذا المنتج رائع جدًا وساعدني كثيرًا في حياتي اليومية. أنصح الجميع باستخدامه.

Summary:
 The customer review states that the product is wonderful and has greatly helped them in their daily life. They highly recommend it to others.

Follow-Up Message:
 Thank you for sharing your positive experience with our product! We are thrilled to hear that it has made such a difference in your daily life. If you need any further assistance or have any questions, please don't hesitate to reach out to us. We are here to help and ensure your satisfaction with our products. Thank you for recommending us to others!


In [42]:
# Spanish review example
spanish_review = "Este producto superó mis expectativas. Lo recomiendo a todos sin dudarlo."
result = overall_chain.invoke({
    "review": spanish_review,
    "target_language": "Spanish"
})

print("Translated Review:\n", result["translated_review"])
print("\n Summary:\n", result["summary"])
print("\n Follow-Up Message:\n", result["follow_up"])





> Entering new SequentialChain chain...

> Finished chain.
Translated Review:
 This product exceeded my expectations. I recommend it to everyone without hesitation. 

Translation: Este producto superó mis expectativas. Lo recomiendo a todos sin dudarlo.

 Summary:
 The customer was very satisfied with the product and highly recommends it to others without any hesitation.

 Follow-Up Message:
 Dear customer,

Thank you for taking the time to share your positive feedback with us. We are thrilled to hear that the product exceeded your expectations and that you would recommend it to others without hesitation.

If you have any questions or need further assistance in the future, please do not hesitate to reach out to us. We are here to help and ensure that you have a great experience with our products.

Thank you for choosing us!

Sincerely, 
[Your Name] 
Customer Service Team
